In [ ]:
import torch
print(torch.cuda.current_device())

In [6]:
!python --version

In [9]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Apr 21 19:42:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/

In [ ]:
# !conda install pytorch torchvision -c pytorch --yes
# !pip install torch torchvision

In [ ]:
# !python -m pip install  torch transformers
# !pip install torch torchvision transformers
# !conda install pytorch torchvision  -c pytorch


In [ ]:
# !which pip
# !conda install -c conda-forge python=3.7 python pip3
# !conda install -c conda-forge transformers
# !conda install pytorch torchvision --yes

# !pip uninstall transformers --yes
# #!pip uninstall torch --yes
# !pip install transformers
# !pip install torch --yes
# # !pip install transformers --yes
# # !pip install cython --yes


# from datasets import load_dataset
# datase = load_dataset("json", data_files="te.json")


In [1]:
import json
import math
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
import argparse
import logging
from torch.utils.data import IterableDataset
import gzip
import json
import sys
from datetime import datetime
import random
from shutil import copyfile
import os
import re
# from huggingface_hub import Repository
# repo = Repository("gpt2-reddit", clone_from="skunusot/gpt2-reddit")
# repo.git_pull()

output_dir_main = "./reddit-model"


parser = argparse.ArgumentParser()
parser.add_argument("--model_name", default="gpt2")
parser.add_argument("--train_files", required=False, nargs='+', default=['./combinedcleaned-full.jsonl.gz'])
parser.add_argument("--name", required=False, default="reddit")
parser.add_argument("--train_size", default=320000, type=int)
parser.add_argument("--eval_size", default=50, type=int)
parser.add_argument("--test_size", default=80000, type=int)
args = parser.parse_args(args=[])

2023-04-23 18:05:17.455250: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 18:05:23.112726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

def get_example(s):
        raw_example = json.loads(s)
        return [(clean_title(raw_example['title'])), clean_title(raw_example['body'])]
    
def clean_title(text):
    text = text.replace("&amp;", "&").strip()
    if text.startswith("["):
        text = re.sub("^\[[a-zA-Z0-9]+\]", "", text).strip()

    if text.endswith("]"):
        text = re.sub("\[[a-zA-Z0-9\.]+\]$", "", text).strip()

    if text.startswith("/r"):
        text = re.sub("^/[a-zA-Z0-9/]+[;,: \-]+", "", text).strip()

    return text
    
class RedditTitleDataset(IterableDataset):
    def __init__(self, filepath):
        self.filepath = filepath
        self.examples = []
    
    def __iter__(self):
        with gzip.open(self.filepath, 'rt', encoding='utf-8') as fIn:
            for line in fIn:
                example = get_example(json.loads(line))
                if example is not None and example[0] != '' and example[1] != '':
                    self.examples.append(example)
                    yield example

    def get_total(self):
        total = 0
        with gzip.open(self.filepath, 'rt', encoding='utf-8') as fIn:
            for line in fIn:
                example = get_example(json.loads(line))
                if example[0] != '' and example[1] != '':
                    total += 1
        
        return total
        
    
    def __len__(self):
        return len(self.examples)

device = torch.device("cuda")

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(args.model_name)

# if tokenizer.pad_token is None:
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': tokenizer.bos_token, 'eos_token': tokenizer.eos_token})

# Define data collator
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=False
# )
def data_collator(examples):
    # print(examples)
    if examples is None: 
        return None
    targets = [row[0] for row in examples]
    inputs = [row[1] for row in examples]
    # print("targets: ")
    # print(targets)
    # print("inputs: ")
    # print(inputs)
    model_inputs = tokenizer(targets[0], inputs[0], padding=True, truncation=True, return_tensors='pt', pad_to_multiple_of=8)

    if not model_inputs.data["input_ids"] is None or len(model_inputs.data["input_ids"]) == 0:
        # print("model_inputs")
        # print(model_inputs.data["input_ids"])
        model_inputs["labels"] = torch.tensor(model_inputs.data["input_ids"]).to(device)
    return model_inputs

    

import gc

# Create dataset and dataloader
reddit_dataset = RedditTitleDataset(args.train_files[0])
reddit_dataset_iter = iter(reddit_dataset)
# train_dataset = [next(reddit_dataset_iter) for _ in range(reddit_dataset.get_total() - args.test_size - args.eval_size -5200087)]

# print(reddit_dataset.get_total())
# reddit_dataset.get_total() - args.eval_size-100000
train_dataset = [next(reddit_dataset_iter) for _ in range(470000)] #5200087
eval_dataset = [next(reddit_dataset_iter) for _ in range(args.eval_size)]
del reddit_dataset
del reddit_dataset_iter

gc.collect()


print("Target:", eval_dataset[50][0])
print("Input:", eval_dataset[50][1])





open ./combinedcleaned-full.jsonl.gz
Target: The inverse solubility of certain cellulosic rheology modifiers, at room temperature they are a liquid, but as you heat them up they harden and form a solid.
Input: I've actually worked with that. I was working on a hyaluronic acid/methylcellulose hydrogel that was both thermally setting (solidifies at 37 C) and shear thinning (liquefies when pushed through a syringe). We were looking at usin it for deliverying a hydrogel scaffold into a body through a syringe, and having it solidify in place.


In [4]:
print("Target:", eval_dataset[51][0])
print("Input:", eval_dataset[51][1])

Target: Just like last year, we are not doing any April Fool's day jokes, nor are we allowing them. Please do not submit anything like that. We are also not doing a regular AMA (because it would not be fair to a guest to do an AMA on April first.) We are taking this opportunity to have a discussion with the community. What are we doing right or wrong? How could we make /r/science better? Ask us anything.
Input: If the comment section remains active it doesn't get archived, that's how that longest comment post keeps rolling.


In [35]:

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir_main,
    evaluation_strategy="steps",
    eval_steps=250,
#     save_total_limit=2,
    learning_rate=1e-5,
    # fp16=True,
    per_device_train_batch_size=256,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=256,
    num_train_epochs=4,
    weight_decay=0.01,
    hub_strategy="every_save",
    save_strategy="no",
    push_to_hub=False,
    hub_model_id="skunusot/gpt2-reddit",
    hub_private_repo=True,
    hub_token="whf_LbwUQBNXqnUndGiCJePZLvNzcVRQCOXtSI",
    dataloader_pin_memory=False
)

import evaluate
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# # implement the compute matrix to be used in evaluation the model
# def compute_metrics(pred):
#     labels_ids = pred.label_ids
#     pred_ids = pred.predictions

#     # all unnecessary tokens are removed
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     labels_ids[labels_ids == -100] = tokenizer.pad_token_id
#     label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
#     rouge_score = rouge.compute(predictions=pred_str, references=label_str),

#     return {"bleu": round(corpus_bleu(pred_str, [label_str]).score, 4),
#             "rouge": rouge_score,
#            }


model = GPT2LMHeadModel.from_pretrained(args.model_name).to(device)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
#     compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()
# # trainer.push_to_hub()

trainer.save_model(output_dir_main)
tokenizer.save_pretrained(output_dir_main)
# compute_scores(test_dataset)

# def compute_scores(test_dataset):
#     tokenizer = GPT2Tokenizer.from_pretrained(output_dir_main)
#     model = GPT2LMHeadModel.from_pretrained(output_dir_main)
#     model.eval()
#     test_input_ids = tokenizer.batch_encode_plus(test_dataset, padding=True, truncation=True, return_tensors='pt')['input_ids']

#     with torch.no_grad():
#         outputs = model(test_input_ids, labels=test_input_ids)
#         loss = outputs[0]

#     # Compute the perplexity as the ex
#     ponential of the cross-entropy loss
#     perplexity = math.exp(loss)
#     print(f"Perplexity: {perplexity:.2f}")    

    
    




/scratch/12244466/ipykernel_16745/235063363.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_inputs["labels"] = torch.tensor(model_inputs.data["input_ids"]).to(device)


Step,Training Loss,Validation Loss
250,No log,3.558043
500,3.987300,3.560935
750,3.987300,3.560512


('./reddit-model/tokenizer_config.json',
 './reddit-model/special_tokens_map.json',
 './reddit-model/vocab.json',
 './reddit-model/merges.txt',
 './reddit-model/added_tokens.json',
 './reddit-model/tokenizer.json')

In [ ]:
evaultion = trainer.evaluate()
print(evaultion)

from transformers import AutoModelWithLMHead, AutoTokenizer, StoppingCriteriaList, MaxLengthCriteria

stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=64)])

def generate_text(model, tokenizer, prompt, length=200, temperature=1.0):
    inp = tokenizer(prompt, return_tensors="pt")
    input_ids = inp["input_ids"]
    a = inp["attention_mask"]
    output = model.generate(
        input_ids,
        attention_mask=a,
        do_sample=True,
#         max_length=length + len(input_ids[0]),
        temperature=temperature,
        top_k=50,
        top_p=0.95,
        # repetition_penalty=1.2,
        # repetition_penalty_range=512,
        # repetition_penalty_slope=3.33,
        pad_token_id=50256,
        eos_token_id=50256
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


# tokenizer = GPT2Tokenizer.from_pretrained(output_dir_main)
# model = GPT2LMHeadModel.from_pretrained(output_dir_main)


# tokenizer = AutoTokenizer.from_pretrained(args.model_name)
# model = AutoModelWithLMHead.from_pretrained(args.model_name)
tokenizer = AutoTokenizer.from_pretrained(output_dir_main)
model = AutoModelWithLMHead.from_pretrained(output_dir_main)

model.eval()
# model = torch.load(model_path)
prompt = "How to use Excel?"
# prompt = "What do you know about ethics?"
# prompt = "Do we need to punish people for reckless driving?"
# prompt ="Do you know Thomas Massie?"
generated_text = generate_text(model, tokenizer, prompt)
print(generated_text)




inp = tokenizer(prompt, return_tensors="pt")
input_ids = inp["input_ids"]
a = inp["attention_mask"]
print("Output:\n" + 100 * '-')

output = model.generate(input_ids=input_ids, attention_mask=a, max_new_tokens = 150,  do_sample=True,pad_token_id=50256,
        eos_token_id=50256)
generated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_sentence)

beam_outputs = model.generate(
    input_ids, 
    attention_mask=a,
    num_beams=5, 
    early_stopping=True,
    do_sample=True,
    num_return_sequences=5, 
#     max_length=50 + len(input_ids[0]),
#     temperature=1,
    top_k=50,
    top_p=0.95,
    no_repeat_ngram_size=2
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_outputs[0], skip_special_tokens=True))

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))



